## Тестирование программ

После написания программы вы можете вручную исследовать ее поведение в различных сценариях. Делая это, программист тестирует написанный код в ручном режиме. Это так называемые исследовательские ручные тесты. Если после во время таких тестов программист обнаруживает ошибочное поведение программы, он пытается его исправить внесением правок в соответствующий участок кода. После этого он снова проводит ручное тестирование. И его код снова выдает ошибку &mdash; еще хуже, если уже в другом месте. Поиск ошибок в программе &mdash; это изнурительный процесс, а повторение одних и тех же действий по тестированию делают этот процесс еще более невыносимым.

К счастью, есть удобное решение &mdash; это подготовка набора автоматических тестов. Единожды подготовив автоматические тесты, программист более не тратит свое время на ручной ввод тестовых данных при каждом найденном баге. Он просто запускает все тесты сразу одной командой и смотрит на вывод неудачных тестов. На анализ удачных тестов программист при этом вообще не тратит времени.

Автоматические тесты принципиально можно поделить на две группы: интеграционные (integration) и модульные (unit) тесты. Представьте, что вы хотите проверить работу фар в автомобиле. Для этого вы включаете фары с помощью переключателя, а затем выходите из машины и смотрите на фары &mdash; и они не работают. В чем может быть дело? Неполадки могут быть в проводке в машине, в целостности лампы, в заряде аккумулятора. Непредсказуемое поведение может быть даже попросту в том, что между включением фар и тем, как вы на них смотрите, проходит порядка 10 секунд. В такой ситуации вы пытаетесь проверить систему, состоящую из множества компонентов &mdash; она не работает, но вы не можете сразу сказать, в чем дело. Такую проверку можно сравнить с интеграционным тестированием.

Модульное тестирование подразумевает проверку отдельного компонента системы, что позволяет локализовать неисправность. Например, ручной модульный тест может включать в себя исследование проводки, лампочки, заряда аккумулятора с помощью мультиметра. Автоматический модульный тест может быть создан разработчиками автомобиля &mdash; они добавят на панель приборов индикатор для датчика исправности лампы в фаре.

### Assertions для проверки логических выражений

В программировании в роли датчиков исправности выступают *assertions* (утверждения). Они позволяют понять, является ли данное выражение истинным в заданных условиях. В языке python assertions задаются синтаксисом
```python
assert bool_expr, assertion_msg
```
где ```bool_expr``` &mdash; это логическое выражение, истинность которого необходимо проверить, ```assertion_msg``` &mdash; это опциональная строка, которую выведет программа, если выражение ```bool_expr``` оказалось неверным. В этом случае интерпретатор выдает ошибку ```AssertionError``` с сообщением ```assertion_msg```. Логическое выражение в assertion может быть неверным только если в коде есть баг. Другими словами, программист никогда не должен увидеть ```AssertionError``` в работающей программе.

Assertions полезны при разработке и отладке кода, но на них нельзя надеяться при использовании программы в нормальных условиях. Вот некоторые из сценариев, при которых assertions могут использоваться: документирование и тестирование кода, отладка багов.

Например, рассмотрим код для работы с геометрической фигурой &mdash; кругом в разработке.

In [ ]:
from math import pi

class Circle:
    def __init__(self, radius):
        if radius < 0:
            raise ValueError("positive radius expected")
        self.radius = radius

    def area(self):
        assert self.radius >= 0, "positive radius expected"
        return pi * self.radius ** 2

Здесь конструктор класса ```__init__``` запрещает отрицательный радиус круга, функция расчета площади ```area``` проверяет, что радиус неотрицателен. Возникает вопрос, зачем функция ```area``` об этом заботится, если конструктор уже содержит в себе необходимые условия, и у пользователя не получится создать круг с отрицательным радиусом? Для ответа на этот вопрос попросим другого программиста написать функцию ```correct_radius``` для умножения радиуса колеса на некоторый коэффициент.

In [ ]:
class Circle:
    def __init__(self, radius):
        if radius < 0:
            raise ValueError("positive radius expected")
        self.radius = radius

    def area(self):
        assert self.radius >= 0, "positive radius expected"
        return pi * self.radius ** 2

    def correct_radius(self, correction_coefficient):
        self.radius *= correction_coefficient

А теперь воспользуемся нашей программой.

In [ ]:
tire = Circle(42)
tire.area()

In [ ]:
tire.correct_radius(-1.02)
tire.area()

Мы получили ошибку только благодаря осмотрительности первого программиста, который явно учел возможность того, что кто-то когда-нибудь сделает радиус отрицательным. 

На данном этапе кому-то захочется обернуть ```assert``` в блок ```try ... except```, но делать это категорически нельзя. Повторим, что assertions актуальны только при разработке программы, и могут быть проигнорированы при подготовке пакетов для реального использования. Задача программиста &mdash; это не обработать возникающие ```AssertionError```, а сделать так, чтобы при всюду существующих ```assert``` ошибки этого типа никогда не возникали.

Приведенный выше пример показывает использование assertions для отладки багов. Программист использовал так называемые preconditions &mdash; он разместил ```assert``` в блоке верификации входных данных. То же самое можно делать для проверки возвращаемых из функции значений перед ```return```. В этом случае такие выражения называются postconditions.

Одновременно с отладкой багов, мы добавили в код документацию. Прочитав ```assert self.radius >= 0```, опытный программист сразу поймет, что в этой функции имеет дело с неотрицательным значением радиуса. Такое документирование наряду с качественными названиям переменных оказывается более действенным, чем добавление строковых комментариев вроде:
```python
# The radius must be non-negative.
```
Такие комментарии, к сожалению, не гарантируют ничего. 

Напомним, что для работы с assertions нужно использовать базовый синтаксис
```python
assert expression, assertion_message
```

In [ ]:
number = 10
assert isinstance(number, (int, float))
assert number > 0, "number should be > 0"

Ошибки не возникло, так как все выражения в ```assert``` оказались истинными.

In [ ]:
number = -10
assert isinstance(number, (int, float))
assert number > 0, "number should be > 0"

Получили ошибку, так как выражение в последнем ```assert``` оказалось ложным.

Далее перечислим основные сценарии использования assertions. **Попробуйте самостоятельно предсказать вывод ячейки перед ее запуском. Обратите внимание на то, какая строка вызвала ошибку.**

* Утверждения сравнения (comparison assertions)

In [ ]:
assert 3 > 2
assert 3 < 2

In [ ]:
assert 3 == 2 

In [ ]:
assert 3 > 2 and 4 > 3
assert 3 < 2 or 4 < 3

* Утверждения вхождения в множество (membership assertions)

In [ ]:
numbers = [1, 2, 3, 4]
assert 2 in numbers
assert 10 in numbers

In [ ]:
assert not 10 in numbers

* Утверждения идентичности (identity assertions)

In [ ]:
x = 1
y = x
z = None

assert x is y
assert z is None
assert y is not x

* Утверждения типа (type assertions)

In [ ]:
num = 10.0
assert isinstance(num, int)

In [ ]:
assert isinstance(num, float)
assert isinstance(num, (int, float))

### Assertions не подходят для production

Теперь разберемся, почему на assertions нельзя положиться нигде, кроме разработки и тестирования. Assertions можно отключить на этапе использования, применив флаг -O при выполнении программы (```python -O main.py```). В этом случае переменная ```__debug__``` принимает значение ```False```, что означает оптимизированное исполнение программы.

Рассмотрим выражение
```python
if __debug__:
    if not expression:
        raise AssertionError(assertion_message)
```
оно эквивалентно выражению
```python
assert expression, assertion_message
```
Поэтому выключение режима отладки (когда ```__debug__ == False```) приводит к тому, что все assertions игнорируются. Это делает бессмысленным обработку и валидацию данных с помощью assertions. В этом случае стоит пользоваться конструкцией ```try ... except ... raise```.

## Использование ```assert``` в написании тестов

Разобравшись с конструкцией ```assert```, мы можем начать использовать ее для написания тестов. Рассмотрим примитивную функцию для подсчета суммы элементов списка ```custom_sum```.

In [7]:
def custom_sum(*args):
    s = 0
    for v in args:
        s += v
    return s

Базовый тест мог бы выглядеть так:

In [ ]:
if custom_sum(1, 2, 3, 4, 5) != 15: 
    print("should be 15")

Использование ```assert``` позволяет выполнить ту же проверку более емко. Более того, тестов может быть несколько.

In [ ]:
assert custom_sum(1, 2, 3, 4, 5) == 15, "should be 15"
assert custom_sum() == 0, "should be 0"

Напишем функцию ```test_custom_sum```, проводящую необходимые тесты.

In [ ]:
def test_custom_sum():
    assert custom_sum(1, 2, 3, 4, 5) == 15, "should be 15"
    assert custom_sum() == 0, "should be 0"
    assert custom_sum(0) == 0, "should be 0"
    assert custom_sum(1, -1) == 0, "should be 0"

Теперь функцию необходимо вызвать. 

In [ ]:
test_custom_sum()

Мы не получаем никаких ошибок, потому что функция реализована в соответствии с описанными тестовыми случаями.

## Использование ```unittest``` в написании тестов

Для каждой &quot;настоящей&quot; функции нужно писать собственную &quot;тестовую&quot;, затем еще нужно не забыть ее вызвать. Было бы удобнее автоматизировать и этот процесс. Для этого пользуются пакетами для автотестирования: unittest, nose2, pytest. Первый из них сейчас входит в состав стандартной библиотеки python. Остальные делают небольшие надстройки над unittest, добавляют некоторый функционал. Поскольку эти возможности не представляют значительного учебного интереса, мы остановимся на самом базовом пакете &mdash; unittest.

Ремарка: для работы unittest в jupyter-notebook необходимо установить дополнительный пакет. Сделать это можно, выполнив следующую ячейку. Если далее вы планируете пользоваться отдельными .py-файлами, вам это действие выполнять не требуется.

In [1]:
# The next line can be REMOVED in case you are using .py-file
%pip install ipython_unittest

Note: you may need to restart the kernel to use updated packages.


Импортируем unittest. Здесь возникает вспомогательная команда, которую можно пропустить, если вы не пользуетесь jupyter-notebook.

In [5]:
import unittest

# The next line can be REMOVED in case you are using .py-file
%reload_ext ipython_unittest

Напишем сам тест. Здесь для реализации одного тестового случая нужно создать унаследованный от ```unittest.TestCase``` класс. Его следует назвать ```TestXXX```, где ```XXX``` обозначает название тестового случая. Далее в классе создается метод с названием ```test_xxx```, где ```xxx``` обозначает название тестируемой функции. Метод ```test_xxx``` наполняется логикой тестов.

Вместо встроенной конструкции ```assert``` в unittest используются методы вида ```.assertYYY```. Далее приведены основные возможности этих методов.

| unittest                      | pure python                                |
|-------------------------------|--------------------------------------------|
| ```.assertEqual(a, b)```      | ```assert a == b```                        |
| ```.assertTrue(a)```          | ```assert a is True```                     |
| ```.assertIs(a, b)```         | ```assert a is b```                        |
| ```.assertIn(a, b)```         | ```assert a in b```                        |
| ```.assertIsInstance(a, b)``` | ```assert isinstance(a, b)```              |
| ```.assertNotEqual(a, b)```   | ```assert not a == b (or assert a != b)``` |

Последний метод с ```Not``` приведен для примера, он доступен для всех перечисленных &quot;положительных&quot; методов.

In [8]:
%%unittest_main
# The previous line can be REMOVED in case you are using .py-file

class TestOrdinaryList(unittest.TestCase):
    def test_custom_sum(self):
        self.assertEqual(custom_sum(1, 2, 3, 4), 10, "should be 10")

class TestEmptyList(unittest.TestCase):
    def test_custom_sum(self):
        self.assertEqual(custom_sum(), 0, "should be 0")

class TestListOfZeros(unittest.TestCase):
    def test_custom_sum(self):
        self.assertEqual(custom_sum(0, 0, 0), 0, "should be 0")

class TestListWithNegatives(unittest.TestCase):
    def test_custom_sum(self):
        self.assertEqual(custom_sum(-1, -2, 2, 3), 2, "should be 2")

class TestAlwaysFAIL(unittest.TestCase):
    def test_custom_sum(self):
        self.assertEqual(custom_sum(), 10, "should be 2")

Fail

....F
FAIL: test_custom_sum (__main__.TestAlwaysFAIL.test_custom_sum)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 21, in test_custom_sum
AssertionError: 0 != 10 : should be 2

----------------------------------------------------------------------
Ran 5 tests in 0.001s

FAILED (failures=1)


<unittest.runner.TextTestResult run=5 errors=0 failures=1>

Разберем выходные данные. Строка (FAIL) показывает последний вывод ячейки. Далее начинается вывод системы тестирования. В ней первая строка (....F) обозначает, что было выполнено 5 тестов, из них 4 завершились успешно (обозначены по порядку точками), затем 1 тест завершился со статусом FAIL. На следующих строках показаны детали каждого безуспешно завершившегося теста: название тестовой функции, название модуля и тестового кейса. Далее идет Traceback с подробной информацией о причине ошибки.

В случае использования .py-файлов запуск тестов проводится следующим образом. Необходимо выполнить одно из двух действий:
* Добавить в область ```if __name__ == "__main__"``` программы строку ```unittest.main()``` и запустить командой ```python3 <test_filename>.py```;
* Запустить программу командой ```python3 -m unittest discover```, ничего не добавляя в текст программы, ```discover``` означает, что нужно найти все ```test*.py``` файлы в текущей директории.

Придумывать тесты трудно. Как можно к ним подступиться? Задайте себе несколько вопросов:
* Что нужно протестировать?
* Какие входные данные могут быть?
* Какие при этом должны быть выходные данные?
На примере с функцией суммирования, можно отвечать следующим образом: нужно протестировать функцию суммирования; входными данными может быть пустой список, список из одного элемента, список из нескольких элементов; выходными данными должен быть ноль, само число и сумма всех числе, соответственно. Далее необходимо эти слова формализовать в виде теста.

В какой-то момент программист должен подумать и о типе входных данных, чтобы затем их верифицировать. Допустим, в нашей функции суммирования мы уверены только в работоспособности на числах (целых и с плавающей точкой). Проверим, как при этом реагирует наша функция ```custom_sum``` на входные данные вида строка или на смесь типов данных.

In [9]:
custom_sum("abc", "bcda")

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [10]:
custom_sum(10, "abc", "bcda")

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

Если посмотреть на Traceback, то становится понятно, что функция начала свое выполнение и выдала ошибку, как только не справилась с операциями. Несмотря на то, что мы ожидаем такую же ошибку, нам следует ожидать ее гораздо раньше &mdash; на этапе верификации входных данных.

In [12]:
def custom_sum(*args):
    for v in args:
        # Which is better?

        # This:
        if not isinstance(v, (int, float)):
            raise TypeError

        # Or this?
        assert isinstance(v, (int, float))

    s = 0
    for v in args:
        s += v
    return s

Ответ на вопрос в примере зависит от обстоятельств. Самое важное обстоятельство &mdash; это стадия разработки. В большинстве ситуаций лучше сделать следующим образом.

In [13]:
def custom_sum(*args):
    for v in args:
        if not isinstance(v, (int, float)):
            raise TypeError(f"custom_sum does not support type {type(v)} in arguments")

    s = 0
    for v in args:
        s += v
    return s

In [28]:
%%unittest_main
# The previous line can be REMOVED in case you are using .py-file

class TestListOfIncorrectArgs(unittest.TestCase):
    def test_custom_sum(self):
        with self.assertRaises(TypeError):
            custom_sum(1, "2", 3, "4")

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Упражнение 1.

Написать тесты программы для разложения числа на простые множители из упражнения 2 в [семинаре 3](https://github.com/Klimkou/CS_SEPMP_2023/blob/main/Seminars/Seminar_3.ipynb).

In [29]:
def f(n):
    if n == 0:
        return 0
    i = 2
    a = []
    while i <= (n**0.5)+1:
        while n % i == 0:
            a.append(i + f(n-n))
            n = n // i
        i = i + 1
    if n > 1:
        a.append(n)
    return a


def func(a, b):
    assert isinstance(a, int) and isinstance(b, int), 'a, b, must be int'
    assert a <= 0 and b <= 0, '<=zero'
    d1 = f(a)
    d2 = f(b)
    m = []
    for i in d1:
        if i in d2:
            m.append(i)
            d2.pop(d2.index(i))
    ans = 1
    for j in m:
        ans *= j
    for i in range(100):
        y = (ans - a*i)/b
        y1 = (ans + a*i)/b
        if y == int(y):
            return ((i, (ans - a*i)//b, ans))
            break
        if y1 == int(y1):
            return (-i, (ans + a*i)//b, ans)
            break


def test_custom_sum():
    assert func(1, 2) == (1, 0, 1), "should be (1, 0, 1)"
    assert func(2, 3) == (-1, 1, 1), "should be (-1, 1, 1)"
    assert func(3, 4) == (-1, 1, 1), "should be (-1, 1, 1)"


AssertionError: <=zero

## Упражнение 2.

Написать тесты программы для расчета коэффициентов МНК из упражнения 6 в [семинаре 3](https://github.com/Klimkou/CS_SEPMP_2023/blob/main/Seminars/Seminar_3.ipynb).

In [38]:
import numpy as np


def f(x, y):
    if not (isinstance(x, list) and isinstance(y, list)):
        raise TypeError
    x = np.array(x)
    y = np.array(y)
    n = len(x)
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    sum_xy = np.sum(x * y)
    sum_x2 = np.sum(x**2)
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    intercept = (sum_y - slope * sum_x) / n
    return slope, intercept


In [39]:
%%unittest_main


class TestOrdinaryList(unittest.TestCase):
    def test_f(self):
        self.assertEqual(f([0, 1, 1], [0, 1, 1]),
                         (np.float64(1.0), np.float64(0.0)), "wrong answer")


class TestIncorrectArg(unittest.TestCase):
    def test_f(self):
        with self.assertRaises(TypeError):
            f("2", "4")


Success

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

## Упражнение 3.

Написать тесты программы, которая реализует быструю сортировку (Тони Хоара).

In [44]:
import random


def quick_sort(a):
    if not isinstance(a, list):
        raise TypeError
    if len(a) <= 1:
        return a
    else:
        q = random.choice(a)
        ll = []
        m = []
        r = []
        for elem in a:
            if elem < q:
                ll.append(elem)
            elif elem > q:
                r.append(elem)
            else:
                m.append(elem)
        return quick_sort(ll) + M + quick_sort(r)


In [46]:
%%unittest_main


class TestOrdinaryList(unittest.TestCase):
    def test_quick_sort(self):
        self.assertEqual(quick_sort([5, 4, 2, 1]),
                         [1, 2, 4, 5], "wrong answer")


class TestIncorrectArg(unittest.TestCase):
    def test_quick_sort(self):
        with self.assertRaises(TypeError):
            f(2)


Success

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

## Упражнение 4.

Написать тесты программы для дешифровки текста, закодированного шифром Цезаря, из [семинара X](https://github.com/Klimkou/CS_SEPMP_2023/blob/main/Seminars/Seminar_X.ipynb) 

In [49]:
class Caesar:
    alphabet = "яюэьыъщшчцхфутсрпонмлкйизжёедгвба"

    def __init__(self, key):
        self._encode = dict()
        for i in range(len(self.alphabet)):
            letter = self.alphabet[i]
            encoded = self.alphabet[(i + key) % len(self.alphabet)]
            self._encode[letter] = encoded
            self._encode[letter.upper()] = encoded.upper()
        self._decode = dict()
        for i in range(len(self.alphabet)):
            letter = self.alphabet[i]
            encoded = self.alphabet[(i - key) % len(self.alphabet)]
            self._decode[letter] = encoded
            self._decode[letter.upper()] = encoded.upper()

    def encode(self, text):
        return ''.join([self._encode.get(char, char) for char in text])

    def decode(self, text):
        return ''.join([self._decode.get(char, char) for char in text])


def czr(line):
    if not isinstance(line, (str, int)):
        raise TypeError
    key = 2
    cipher = Caesar(key)
    return (cipher.decode(line))


In [51]:
%%unittest_main


class TestOrdinaryList(unittest.TestCase):
    def test_czr(self):
        self.assertEqual(czr('йцй'), 'лшл', "wrong answer")


class TestIncorrectArg(unittest.TestCase):
    def test_czr(self):
        with self.assertRaises(TypeError):
            czr([4, 4])


Success

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>